In [1]:
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Bloco de convolução com identidade
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        '''
        O shortcut é uma conexão de atalho que é adicionada a uma camada convolucional em uma ResNet (rede neural residual).
        Ele é utilizado para ajudar a resolver o problema de degradação do desempenho da rede quando a profundidade da rede é aumentada.

        Quando a profundidade da rede é aumentada, o desempenho do treinamento pode começar a piorar, já que é difícil para a rede 
        aprender a identificar os recursos relevantes em camadas mais profundas. O shortcut ajuda a resolver este problema, permitindo
        que as informações sejam passadas diretamente para camadas mais profundas sem passar por várias camadas intermediárias. 
        Isso permite que a rede aprenda com mais eficiência a identificar os recursos relevantes em camadas mais profundas, 
        melhorando o desempenho da rede.
        '''

        self.shortcut = nn.Sequential()

        '''
        O shortcut (também conhecido como conexão de salto) é adicionado em uma ResNet quando a entrada e a saída de uma camada não têm 
        o mesmo tamanho espacial, o que pode acontecer por diferentes motivos, como quando a stride é diferente de 1 ou os canais de entrada
        e saída são diferentes. Isso é necessário para garantir que o sinal de ativação possa fluir através da rede neural sem ser diminuído
        ou aumentado drasticamente em dimensões espaciais.

        Sem a conexão de salto, a informação da entrada pode ser "perdida" em uma camada com stride maior que 1 ou em camadas convolucionais
        com números diferentes de canais de entrada e saída. A adição do shortcut permite que a informação da entrada seja preservada em todas
        as camadas da ResNet e combinada com a saída da camada posterior. Essa abordagem também ajuda a resolver o problema do desvanecimento
        do gradiente em redes muito profundas, permitindo que o gradiente flua mais facilmente através da rede.
        '''

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [3]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Conv2d(
            3, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out